In [1]:
!pip install flask-ngrok
!pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18894 sha256=c9945eff4d7ba9675db2153782f235aad940127dde10c1ed62b0c628fe068cbb
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras-self-attention


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Concatenate, Flatten, Dense, Attention
from tensorflow.keras.models import Model
from keras.layers import Attention
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import string
from flask import Flask, render_template, redirect, request, jsonify
from flask_ngrok import run_with_ngrok
import keras_self_attention
from tensorflow.keras import layers
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Sequential
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/chatbot_json.json') as content:
    data1=json.load(content)

In [ ]:
tag=[]
inputs=[]
responses={}

for intent in data1['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
        inputs.append(lines)
        tag.append(intent['tag'])

In [ ]:
data=pd.DataFrame({"inputs":inputs,"tags":tag})
data
data = data.sample(frac=1)

In [ ]:
# Preprocess the text data
import string

data['inputs'] = data['inputs'].apply(lambda wrd: [ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
data

,inputs,tags
274,application for special capital investment sub...,Application_for_Special_Capital_(Investment)_S...
321,assistance for miscarriage or termination of p...,Assistance_for_miscarriage_or_Termination_of_P...
149,reservation for micro small and medium enterp...,"Reservation_for_Micro_,_Small_and_Medium_Enter..."
311,assistance for marriage sponsored,Assistance_for_Marriage_Sponsored
27,students in plusone at reputed schools eligibi...,students_in_plusone_at_reputed_schools_Eligibi...
...,...,...
327,assistance for purchase of spectacles benefici...,Assistance_for_purchase_of_Spectacles_Benefici...
151,reservation for micro enterprises in tansidco ...,Reservation_for_Micro_Enterprises_in_TANSIDCO_...
198,price preference for micro and small enterpris...,Price_Preference_for_Micro_and_Small_Enterpris...
207,free education scheme beneficiaries,FREE_EDUCATION_SCHEME_Beneficiaries


In [ ]:
#Tokenize the Data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])


from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(train)


from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y_train=le.fit_transform(data['tags'])
print(data['tags'])
print(y_train)

274    Application_for_Special_Capital_(Investment)_S...
321    Assistance_for_miscarriage_or_Termination_of_P...
149    Reservation_for_Micro_,_Small_and_Medium_Enter...
311                    Assistance_for_Marriage_Sponsored
27     students_in_plusone_at_reputed_schools_Eligibi...
                             ...                        
327    Assistance_for_purchase_of_Spectacles_Benefici...
151    Reservation_for_Micro_Enterprises_in_TANSIDCO_...
198    Price_Preference_for_Micro_and_Small_Enterpris...
207                  FREE_EDUCATION_SCHEME_Beneficiaries
140    Skill_Development_Training_Schemes_and_Skill_U...
Name: tags, Length: 396, dtype: object
[ 20  54 255  44 378 282  10  64 276 281 183 138 307 258  13 202 262   2
 151  42  59 316 147 340   5 350 270  25 193  17 361 128 117 174 254 268
 196 251 317 239 115  57 347 215  34 337  90 119 310   0 303 379 163  80
 265 375 225 154 156 171 212 226 352 366 294 365 141 252 161 102 104  77
 250 299 181 155  69 107 213 324 139 284  

In [ ]:
input_shape=x_train.shape[1]
print(input_shape)

21


In [ ]:
vocabulary = len(tokenizer.word_index)
print(tokenizer.word_index)
print('number of unquie words',vocabulary)

output_length=le.classes_.shape[0]
print('Output length ',output_length)

{'for': 1, 'and': 2, 'of': 3, 'sponsored': 4, 'eligibility': 5, 'description': 6, 'beneficiaries': 7, 'availability': 8, 'in': 9, 'development': 10, 'scheme': 11, 'cooperative': 12, 'the': 13, 'subsidy': 14, 'or': 15, 'agriculture': 16, 'loan': 17, 'to': 18, 'primary': 19, 'rural': 20, 'bank': 21, 'schemes': 22, 'assistance': 23, 'micro': 24, 'small': 25, 'enterprises': 26, 'on': 27, 'scholarship': 28, 'district': 29, 'level': 30, 'standard': 31, 'education': 32, 'fund': 33, 'production': 34, 'seeds': 35, 'medium': 36, '12th': 37, 'prizes': 38, 'free': 39, 'journalists': 40, 'registration': 41, 'under': 42, 'training': 43, 'farmers': 44, 'certified': 45, 'application': 46, 'special': 47, 'capital': 48, 'industrial': 49, 'estates': 50, 'students': 51, 'public': 52, 'examination': 53, '10th': 54, 'india': 55, 'purchase': 56, 'term': 57, 'obtained': 58, 'award': 59, 'technologies': 60, 'stamp': 61, 'duty': 62, 'sector': 63, 'maize': 64, 'technology': 65, 'integrated': 66, 'miscarriage': 6

In [ ]:
# i=Input(shape=(input_shape,))
# x=Embedding(vocabulary+1,10)(i)
# x=LSTM(10,return_sequences=True)(x)
# x=Flatten()(x)
# x=Dense(output_length,activation='softmax')(x)
# model=Model(i,x)

In [ ]:
#Architecture Model
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

input_shape = x_train.shape[1]
vocabulary_size = len(tokenizer.word_index) + 1
i = Input(shape=(input_shape,))
x = Embedding(vocabulary_size, 10)(i)
x = Conv1D(32, 3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(len(set(y_train)), activation='softmax')(x)
# Define the model
model = Model(i, x)

In [ ]:
model = Model(inputs=i, outputs=x)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=200)

Epoch 1/200
13/13 [==============================] - 2s 4ms/step - loss: 5.9476 - accuracy: 0.0025 
Epoch 2/200
13/13 [==============================] - 0s 3ms/step - loss: 5.9405 - accuracy: 0.0101
Epoch 3/200
13/13 [==============================] - 0s 3ms/step - loss: 5.9374 - accuracy: 0.0076
Epoch 4/200
13/13 [==============================] - 0s 3ms/step - loss: 5.9340 - accuracy: 0.0051
Epoch 5/200
13/13 [==============================] - 0s 3ms/step - loss: 5.9298 - accuracy: 0.0051
Epoch 6/200
13/13 [==============================] - 0s 3ms/step - loss: 5.9243 - accuracy: 0.0051
Epoch 7/200
13/13 [==============================] - 0s 3ms/step - loss: 5.9174 - accuracy: 0.0051
Epoch 8/200
13/13 [==============================] - 0s 3ms/step - loss: 5.9076 - accuracy: 0.0051
Epoch 9/200
13/13 [==============================] - 0s 3ms/step - loss: 5.8948 - accuracy: 0.0051
Epoch 10/200
13/13 [==============================] - 0s 7ms/step - loss: 5.8775 - accuracy: 0.0051
Epoch 11

In [ ]:
from flask import Flask, render_template, redirect, request, jsonify

print(model)

def generate_res(prediction_input):
    texts_p = []

    prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input = ''.join(prediction_input)
    texts_p.append(prediction_input)

    print(prediction_input)

    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)

    output = model.predict(prediction_input)
    output = output.argmax()

    response_tag = le.inverse_transform([output])[0]

    # Assuming 'responses' is a dictionary containing response lists
    if response_tag in responses:
        res = responses[response_tag][0]  # Assuming only one response per tag for simplicity
        print(f"TamilAI ({response_tag}):", res)
        return res
    else:
        return "No response found for the given input."


In [ ]:
from google.colab import output
output.serve_kernel_port_as_window(5000)

<IPython.core.display.Javascript object>

In [ ]:
#Flask
from flask import Flask,render_template,redirect,request,jsonify
from flask_ngrok import run_with_ngrok
templates_folder = '/content/drive/MyDrive/Colab Notebooks/temp'

app=Flask(__name__, template_folder=templates_folder)
app.secret_key="12345"
app.static_folder='static'

@app.route('/')
# def Home():
#     return render_template('Home.html',titleName='TamilAI Chatbot | Main Page')

@app.route('/start')
def startHome():
    return render_template('start.html',titleName='TamilAI Chatbot | Main Page')

@app.route('/learn')
def learnHome():
    return render_template('learn.html',titleName='TamilAI Chatbot | Main Page')

@app.route('/Home',methods=['GET','POST'])
def ChatBot():
    res=0
    if request.method == 'POST':
        message=str(request.form['userReply'])
        bot_response=generate_res(message)
        print('Message  -->',message)

        while True:

           if message!='':
               bot_response = str(bot_response)



               print(bot_response)
               return jsonify({'status':'OK','answer':bot_response})
           elif message == "bye":
               bot_response='Hope to see you soon'
               print(bot_response)
               return jsonify({'status':'OK','answer':bot_response})
               break

    return render_template('Home.html',titleName='TamilAI Chatbot | Chat')

@app.route('/Service')
def Homeservice():
    return render_template('service.html',titleName='TamilAI Chatbot | Main Page')


@app.route('/About')
def HomeAbout():
    return render_template('About.html',titleName='TamilAI Chatbot | Main Page')


@app.route('/Contact')
def HomeContact():
    return render_template('Contact.html',titleName='TamilAI Chatbot | Main Page')


if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/May/2024 09:30:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 09:30:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 09:33:17] "GET /Home HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 09:33:17] "GET /bg.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 09:33:18] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 09:33:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 09:33:21] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:05:41] "GET /learn HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:05:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:05:46] "GET / HTTP/1.1" 200

free education scheme eligibility
1/1 [==============================] - 0s 108ms/step


INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:06:14] "POST /Home HTTP/1.1" 200 -


TamilAI (FREE_EDUCATION_SCHEME_Eligibility): Income:--
Age:--
Community:--
Other Details:Special fee and other non-refundable compulsory fees are paid as prescribed by Government and examination fee in full. Condition No conditions
Message  --> free education scheme eligibility
Income:--
Age:--
Community:--
Other Details:Special fee and other non-refundable compulsory fees are paid as prescribed by Government and examination fee in full. Condition No conditions
sponsored by
1/1 [==============================] - 0s 20ms/step


INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:06:36] "POST /Home HTTP/1.1" 200 -


TamilAI ({scheme_name.replace(' ', '_')}_Sponsored): Sponsored By: {sponsored_by}
Message  --> sponsored by
Sponsored By: {sponsored_by}
free education scheme sponsored by
1/1 [==============================] - 0s 21ms/step


INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:06:52] "POST /Home HTTP/1.1" 200 -


TamilAI (FREE_EDUCATION_SCHEME_Sponsored): Sponsored By: State
Message  --> free education scheme sponsored by
Sponsored By: State


INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:13] "GET /Home HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:14] "GET /bg.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:15] "GET /Service HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:16] "GET /images/html_table.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:17] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:27] "GET /Service HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:28] "GET /images/html_table.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:29] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:30] "GET /Home HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:31] "GET /bg.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 10:07:32] "GET /favicon.ico HTTP/1.1

In [ ]:
while True:
    texts_p = []
    prediction_input = input('You: ')

    prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input = ''.join(prediction_input)
    texts_p.append(prediction_input)

    print(prediction_input)

    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)

    output = model.predict(prediction_input)
    output = output.argmax()

    response_tag = le.inverse_transform([output])[0]

    # Assuming 'responses' is a dictionary containing response lists
    if response_tag in responses:
        res = responses[response_tag][0]  # Selecting the first response for simplicity
        print(f"TamilAI ({response_tag}):", res)
    else:
        print("No response found for the given input.")

    if response_tag == 'goodbye':
        break
